In [1]:
import requests
import pandas as pd

# some Pandas defaults
pd.options.display.max_columns = None

In [2]:
gisaid_df = pd.read_csv(
    '~/Downloads/metadata_tsv_2021_04_26/metadata.tsv', sep='\t')

/Users/julia/miniconda3/envs/deep-learning-nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
gisaid_df.head()

,Virus name,Type,Accession ID,Collection date,Location,Additional location information,Sequence length,Host,Patient age,Gender,Clade,Pango lineage,Pangolin version,Variant,AA Substitutions,Submission date,Is reference?,Is complete?,Is high coverage?,Is low coverage?,N-Content,GC-Content
0,hCoV-19/Australia/NT12/2020,betacoronavirus,EPI_ISL_426900,2020,Oceania / Australia / Northern territory,NaN,29862,Human,unknown,unknown,G,B.1,2021-04-21,NaN,"(NSP15_A283V,NSP12_P323L,Spike_D614G)",2020-04-17,NaN,True,True,NaN,0.006912,0.379674
1,hCoV-19/Australia/NT13/2020,betacoronavirus,EPI_ISL_426901,2020,Oceania / Australia / Northern territory,NaN,29865,Human,unknown,unknown,G,B.1,2021-04-21,NaN,"(NSP15_A283V,NSP12_P323L,Spike_D614G)",2020-04-17,NaN,True,True,NaN,0.008305,0.379554
2,hCoV-19/Australia/NT14/2020,betacoronavirus,EPI_ISL_426902,2020,Oceania / Australia / Northern territory,NaN,29864,Human,unknown,unknown,V,B.40,2021-04-21,NaN,"(NSP14_R163C,NSP3_K38R,NS3_G251V,NSP2_I559V,NS...",2020-04-17,NaN,True,NaN,NaN,0.017929,0.378860
3,hCoV-19/Australia/NT16/2020,betacoronavirus,EPI_ISL_426903,2020,Oceania / Australia / Northern territory,NaN,29813,Human,unknown,unknown,G,B.1.8,2021-04-21,NaN,"(NSP12_P323L,Spike_D614G)",2020-04-17,NaN,True,NaN,NaN,0.009139,0.379244
4,hCoV-19/Australia/NT17/2020,betacoronavirus,EPI_ISL_426904,2020,Oceania / Australia / Northern territory,NaN,29818,Human,unknown,unknown,G,B.1.8,2021-04-21,NaN,"(NSP12_P323L,Spike_D614G)",2020-04-17,NaN,True,True,NaN,0.000436,0.380138


In [4]:
# len(location_set)
location_set = set(gisaid_df.Location.values)
len(location_set)

10766

In [5]:
gisaid_df.Location.value_counts()

Europe / United Kingdom / England                                310516
Europe / United Kingdom / Scotland                                34307
Europe / United Kingdom / Wales                                   33084
Asia / Japan                                                      29078
North America / USA / Texas / Houston                             25968
                                                                  ...  
North America / Panama / Panama Center / Jose Domingo Espinar         1
South America / Argentina / Cordoba / Bell Ville                      1
Europe / Poland / Swietokrzyskie / Zagrody                            1
North America / USA / Georgia / Thomas County                         1
Europe / Poland / Zachodniopomorskie / Stare Brynki                   1
Name: Location, Length: 10766, dtype: int64

In [6]:
def country(location):
    if location.startswith('North America / USA'):
        return 'USA'
    elif location.startswith('South America / Brazil'):
        return 'Brazil'
    elif location.startswith('Asia / India'):
        return 'India'
    elif location.startswith('Africa / South Africa'):
        return 'South Africa'
    elif location.startswith('Europe / United Kingdom'):
        return 'United Kingdom'
    else:
        return ''  # ignore others for now

gisaid_df['country'] = gisaid_df.Location.apply(country)


In [7]:
# gisaid_df.fillna({'AA Substitutions': ''}, inplace=True)
# aa_substitutions = set(gisaid_df['AA Substitutions'].values)

# dels = set()
# for aa_sub_list in aa_substitutions:
#     muts = aa_sub_list.split(',')
#     for mut in muts:
#         if 'del' in mut:
#             dels.add(mut)
# len(dels)
# print(list(dels)[:100])

# def num_snvs(aa_substitutions):
#     muts = aa_substitutions.split(',')
#     return len([x for x in muts if 'del' not in x])
# gisaid_df['num_snvs'] = gisaid_df['AA Substitutions'].apply(num_snvs)

In [8]:
# filter out sequences from the exclusion list
exclude_sequences_url = 'https://raw.githubusercontent.com/nextstrain/ncov/master/defaults/exclude.txt'
exclude_sequences_response = requests.get(exclude_sequences_url)
exclude_sequences = set(
    [x for x in exclude_sequences_response.text.split('\n') if x != '' and not x.startswith('#')])

# prepend "hCoV-19/" to match the GISAID metadata column "Virus name"
exclude_sequences = set(["hCoV-19/%s" % x for x in exclude_sequences])
len(exclude_sequences)

1556

In [9]:
gisaid_df.shape

(1229906, 23)

In [10]:
# drops some sequences as expected
gisaid_df.loc[
    (~gisaid_df['Virus name'].isin(exclude_sequences))
].shape

(1228694, 23)

# Example of GISAID data filtering

## Borrows from some basic outbreak.info and covidcg.org filters

- outbreak.info methods: https://outbreak.info/situation-reports/methods
- covidcg.org methods: https://covidcg.org/?tab=methodology#

### This example uses the following filters:
- sequences at least 20000 in length
- collection date must not be in the future and must be at least year/month, not year alone
- excluding sequences from the Nextstrain exclude list

### We may also want to filter out any sequence that is:
- Less than 29700 bases (vs our 20000 threshold right now)
- greater than 5% ambiguous base calls (N)

In [11]:
# how many B.1.1.7 sequences are in the US? Outbreak.info says 54,109, we're close
# If we were to use this data, we'd filter it using
gisaid_df.loc[
    (gisaid_df['country'] == 'USA') &
    (gisaid_df['Pango lineage'] == 'B.1.1.7') &
    (gisaid_df['Sequence length'] > 20000) &
    (~gisaid_df['Collection date'].isin(['2020', '2021'])) &
    (gisaid_df['Collection date'] < '2021-04-26') &
    (~gisaid_df['Virus name'].isin(exclude_sequences))
].shape

(54117, 23)

In [12]:
# example of still messy location data - there are a few locations in US still not nicely formatted
# as a tree structure
[x for x in location_set if 'USA' in x and not x.startswith('North America / USA')]

['North America/ USA/ Louisiana/ Claiborne Parish',
 'North America/ USA/ Arizona/ Navajo',
 'North America/USA/New York/New York']

# outbreak.info API playing around

In [13]:
# daily b.1.1.7 counts in US
url = 'https://api.outbreak.info/genomics/prevalence-by-location?location_id=USA&cumulative=false&pangolin_lineage=B.1.1.7'
r = requests.get(url)
daily_sum = sum({x['date']: x['lineage_count'] for x in r.json()['results']}.values())
daily_sum

54109

In [14]:
# total b.1.1.7 count in US
url = 'https://api.outbreak.info/genomics/prevalence-by-location?location_id=USA&cumulative=true&pangolin_lineage=B.1.1.7'
r = requests.get(url)
r.json()['results']['lineage_count'] == daily_sum

True

In [15]:
# Compare GISAID and outbreak.info

vocs = ['B.1.1.7', 'B.1.429', 'B.1.427', 'P.1', 'B.1.351']
vois = ['B.1.526', 'B.1.526.1', 'B.1.526.2', 'P.2', 'B.1.617']

In [16]:
def get_gisaid_US_lineage_count(lineage):
    return gisaid_df.loc[
        (gisaid_df['country'] == 'USA') &
        (gisaid_df['Pango lineage'] == lineage) &
        (gisaid_df['Sequence length'] > 20000) &
        (~gisaid_df['Collection date'].isin(['2020', '2021'])) &
        (gisaid_df['Collection date'] < '2021-04-26') &
        (~gisaid_df['Virus name'].isin(exclude_sequences))
    ].shape[0]

def get_outbreak_info_US_lineage_count(lineage):
    url = 'https://api.outbreak.info/genomics/prevalence-by-location?location_id=USA&cumulative=true&pangolin_lineage=%s' % lineage
    r = requests.get(url)
    return r.json()['results']['lineage_count']

counts = []
for lineage in vocs + vois:
    gisaid_lineage_count = get_gisaid_US_lineage_count(lineage)
    outbreak_info_lineage_count = get_outbreak_info_US_lineage_count(lineage)
    pct_difference = (gisaid_lineage_count - outbreak_info_lineage_count) / gisaid_lineage_count
    counts.append({
        'lineage': lineage,
        'GISAID_US_count': gisaid_lineage_count,
        'outbreak_info_US_count': outbreak_info_lineage_count,
        'pct_difference': pct_difference * 100
    })

# Cumulative counts for lineages in US from GISAID and outbreak.info

## (pct_difference is relative to GISAID count)

In [17]:
pd.DataFrame(counts)

,lineage,GISAID_US_count,outbreak_info_US_count,pct_difference
0,B.1.1.7,54117,54109,0.014783
1,B.1.429,21816,21672,0.660066
2,B.1.427,9719,9269,4.630106
3,P.1,2598,2598,0.000000
4,B.1.351,804,804,0.000000
5,B.1.526,10313,10313,0.000000
6,B.1.526.1,4258,4258,0.000000
7,B.1.526.2,3755,3755,0.000000
8,P.2,944,938,0.635593
9,B.1.617,15,15,0.000000
